In [1]:
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import numpy as np
import pprint
pp = pprint.PrettyPrinter(depth=4)


path = './data/road-game-default-rtdb-export.json'

branchs = ['all-games','complete-games']

In [2]:
#load data from the json file
with open(path) as train_file:
    data = json.load(train_file)

In [3]:
branch = branchs[0]
df = pd.DataFrame.from_dict(data[branch], orient='index')
df.reset_index(level=0, inplace=True)
df

,index,additional_comments,aggressively_value,behavior,birthYear,blueScore,computer_value,drivinglicense,education,gender,generously_value,log,press-the-read-instructions-button,redScore,steps,win,wisely_value,predictable_value,answer-the-quiz
0,-MZiWIwdnrn8TJ034z2w,,5,carefulBehavior,1961,20.0,5,valid,BA,male,6,"[None, {'blue': 'from: a6, action: left', 'red...",True,23.0,7.0,True,1,NaN,NaN
1,-MZiWUxZs8BvWglGhc9r,,2,carefulBehavior,1982,19.0,6,valid,BA,female,4,"[None, {'blue': 'from: a6, action: left', 'red...",True,22.0,8.0,True,4,NaN,NaN
2,-MZiWYgxyToIiCSDQY8S,,5,aggressiveBehavior,1953,-105.0,7,valid,BA,female,2,"[None, {'blue': 'from: a6, action: left', 'red...",True,-105.0,5.0,False,6,NaN,NaN
3,-MZiWYz_z_BcQlpReXf_,,6,aggressiveBehavior,1990,34.0,7,valid,Master's degree,male,2,"[None, {'blue': 'from: a6, action: left', 'red...",True,21.0,9.0,True,2,NaN,NaN
4,-MZiWaNZ6n3K3AZDfr-q,,6,aggressiveBehavior,1953,-105.0,6,valid,BA,female,2,"[None, {'blue': 'from: a6, action: left', 'red...",True,-105.0,5.0,False,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,-MbcURCQaLlJFHGxAvyW,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,-MbcbjkKh8M10JRrqfwb,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,-MbcfQpUktm_lYEmxpz2,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,-Mbcz6inzp9NEdJCbK17,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['index', 'additional_comments', 'aggressively_value', 'behavior',
       'birthYear', 'blueScore', 'computer_value', 'drivinglicense',
       'education', 'gender', 'generously_value', 'log',
       'press-the-read-instructions-button', 'redScore', 'steps', 'win',
       'wisely_value', 'predictable_value', 'answer-the-quiz'],
      dtype='object')

In [5]:
# df = df[df['answer-the-quiz'].notnull()]

number of people that start to play:

In [6]:
df['press-the-read-instructions-button'].count()

559

In [7]:
len(df)

678

### blueScore and redScore for each behavior to exel file to do t-test

In [8]:
to_exel = df[['blueScore', 'redScore', 'behavior']]

table_blueScore = pd.DataFrame(index=to_exel.index, columns=to_exel.groupby('behavior').count().index)
for agent in table_blueScore.columns:
    table_blueScore[agent] = to_exel[to_exel['behavior'] == agent]['blueScore']

blueScore_result = pd.concat([table_blueScore[agent].dropna().reset_index().drop('index', axis=1) for agent in table_blueScore.columns], axis=1)

to_exel = df[['blueScore', 'redScore', 'behavior']]

table_blueScore = pd.DataFrame(index=to_exel.index, columns=to_exel.groupby('behavior').count().index)
for agent in table_blueScore.columns:
    table_blueScore[agent] = to_exel[to_exel['behavior'] == agent]['blueScore'] + to_exel[to_exel['behavior'] == agent]['redScore']

bluePlusRedScore_result = pd.concat([table_blueScore[agent].dropna().reset_index().drop('index', axis=1) for agent in table_blueScore.columns], axis=1)

In [9]:
with pd.ExcelWriter('./data/agent_scores.xlsx') as writer:
    blueScore_result.to_excel(writer, sheet_name='blueScore')
    bluePlusRedScore_result.to_excel(writer, sheet_name='blueScore+redScore')

## Survay results:

upload Id workers and search for duplicate workers

In [10]:
path = './data/workers' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    fromOne = pd.read_csv(filename, index_col=None, header=0)
    fromOne['filename'] = filename
    li.append(fromOne)

workers = pd.concat(li, axis=0, ignore_index=True)

In [11]:
df['WorkerId'] = workers['WorkerId']

In [12]:
pd.concat(g for _, g in df.groupby('WorkerId') if len(g) > 1).groupby('WorkerId')['index'].count()

WorkerId
A17AF42SNQNH9C    5
A1AZLJ3R5FQ9B7    2
A1BQI9JKLVCIL0    2
A1K1E4KCVW1HOZ    3
A1KZ21TSAYUHO4    2
A1OW27AZS122X2    2
A1QKW8E5J7NAF2    2
A209KMZQTLEZZC    2
A244IQQR0CTZ3I    2
A2DCKE7398C0XV    2
A2I6ZALE49CVSC    2
A2IFXO71MXB9GK    2
A2OU4N4VBIE91I    2
A2YR29YNFLZSUG    2
A2ZRL1ZWWXJ0L7    3
A30YR5TPOKQYXJ    2
A33XAJW8WGY3O9    2
A34CPKFZXBX1PO    2
A36UD0YBRIWTVL    2
A37XBH865833FE    2
A39ICPRR4OS3OX    2
A3ADHMAWDD7M6V    2
A3GWRDHAURRNK6    2
A3H073G4EPBYOM    2
A3H3BTRV6I3LYU    2
A3KK1BNF5H0N1Z    3
A3L7TVDAMZM0FG    2
A3P35II1RN3FYV    2
A3UWHJTAUTF6TO    2
A6JUD4NCQCDWG     2
ACXP8KHFX06KR     3
AD1ILDUXZHASF     3
ASNMT3I94HZ44     2
ATJ92FYQX176Z     4
Name: index, dtype: int64

We will keep the first one of any 'WorkerId' instance to keep samples unique

In [13]:
df = df.drop_duplicates(subset=['WorkerId'])

# Statistics of the red agent movements:

### Create the statistics table from the log column: 

In [14]:
import flat_table

In [15]:
#To calculate the score:
crushing = -100;
richToDestination = 30;
step = -1;

In [16]:
'''==================================================
Initial set up                              version 1
=================================================='''
#Define all states
all_states=[]
for i in range(12):
    for j in range(12):
            all_states.append((i,j))

#Define rewards for all states
rewards = {}
for (i,j) in all_states:
    if(i == j): # crushed
        rewards[(i,j)] = crushing
    elif(i == 0): # blue agent in a1
        rewards[(i,j)] = richToDestination
    else:
        rewards[(i,j)] = step

#Dictionnary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {}
for (i,j) in all_states:
    if i != 0 and i != j: #if s=(i, j) is not an "end" state
        actions[(i,j)] = ("stay", "left", "down") if (i < 6) else ("up", "stay")

#Define an initial policy
Policy={}
for s in actions.keys():
    Policy[s] = np.random.choice(actions[s])

probs = json.load(open('data/probs1.json',))
    
#Define initial value function 
V={}
for (i,j) in all_states:
    if (i,j) in actions.keys():
        V[(i,j)] = step
    if(i == 0): # blue agent in a1
        V[(i,j)] = richToDestination
    if(i == j): # crushed
        V[(i,j)] = crushing

# Policy evaluation:

In [17]:
def adapt_state(s):
    # takes (i,j) format state and convert it to "a|b[1-6] a|b[1-6]" format
    i = s[0]
    j = s[1]
    if i < 6:
        blueState = 'a' + str(i+1)
    else:
        blueState = 'b' + str(i-5)
    if j < 6:
        redState = 'a' + str(j+1)
    else:
        redState = 'b' + str(j-5)
    return blueState + ' ' + redState

def newPosGivenAction(s, a, agent_i):
    # assert check
    if s[agent_i] == 5 and agent_i == 1:
        actions=["stay"]
    elif s[agent_i] == 0 and agent_i == 0:
        actions=["stay"]
    elif s[agent_i] < 6 and agent_i == 1:
        actions=["right","stay", "down"]
    elif s[agent_i] < 6 and agent_i == 0:
        actions=["left","stay", "down"]
    else:
        actions=["stay", "up"]
    assert a in actions , "action: " + str(a) + " is not vallid at state: " + str(s) + " for agent: " + str(agent_i)
    # agent_i = 0 for blue and 1 for red
    newPos = -1
    if a == "up":
        newPos = s[agent_i]-6
    if a == "down":
        newPos = s[agent_i]+6
    if a == "left":
        newPos = s[agent_i]-1
    if a == "right":
        newPos = s[agent_i]+1
    if a == "stay":
        newPos = s[agent_i]
    assert newPos > -1 and newPos < 12 , "Got out from board!\naction: " + str(a) + " ,state: " + str(s) + " for agent: " + str(agent_i)
    return newPos

def randomProbForAction(s):
    # if aX return 1/3 else (it is bX) return 1/2
    if s[1] == 5: # a6
        return 1
    if s[1] < 6:
        return 1.0/3
    else:
        return 1.0/2
    
def nextStateDistibution(s, a):
    # given state s and action a, return distibution of possibles new states by probs
    nxt = {}
#     print('s: ', s)
#     print('a: ', a)
    bluePossition = newPosGivenAction(s, a, 0)
#     print('bluePossition: ', bluePossition)
    
    # red possible action given state
    if s[1] == 5:
        actions=["stay"]
    elif s[1] < 6:
        actions=["right","stay", "down"]
    else:
        actions=["stay", "up"]
    for r_a in actions:
        redPossition = newPosGivenAction(s, r_a, 1)
#         print('redPossition: '+ str(redPossition))
        newState = (bluePossition, redPossition)
        try:
            nxt[newState] = probs[adapt_state(s)][r_a]
#                 nxt[newState] = {probs[adapt_state(s)][r_a], r_a, adapt_state(newState)}
        except KeyError:
#                 print("Warning! State not found: assume random behavior. ", s)
            nxt[newState] = randomProbForAction(s)
    assert abs(sum(nxt.values()) - 1.0) < 0.0001, "probs not sum to 1.0. \nstate: " + str(s) + " action: " + str(a) + " nxt: " + str(nxt) + "\nsum of probs: " + str(sum(nxt.values()))
    return nxt

In [18]:
#Hyperparameters
SMALL_ENOUGH = 0.0001
GAMMA = 0.99

def policy_eval(policy):
    V={}
    for (i,j) in all_states:
        if (i,j) in actions.keys():
            V[(i,j)] = step
        if(i == 0): # blue agent in a1
            V[(i,j)] = richToDestination
        if(i == j): # crushed
            V[(i,j)] = crushing
    while True:
        biggest_change = 0
        for s in all_states:
            if s in Policy:  
                old_v = V[s]
                nxt = nextStateDistibution(s, policy[s])
                v=0
                #Calculate the value
                for next_state, prob in nxt.items():
                    v += prob * (rewards[s] + GAMMA * V[next_state])
        
                #Save the best of all actions for the state                                
                V[s] = v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        #See if the loop should stop now         
        if biggest_change < SMALL_ENOUGH:
            return V

# evaluate other behaviors:

In [19]:
def randomBehavior():
    Policy={}
    for s in actions.keys():
        Policy[s] = np.random.choice(actions[s])
    return Policy

random_policy = randomBehavior()

In [20]:
randomBehavior_intensity = np.array(list(policy_eval(random_policy).values())).reshape(12,12)
# f, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(randomBehavior_intensity, vmin=-100, vmax=10, annot=True, linewidths=0.1, cmap="YlGnBu")
# ax.set_title('Value function for: randomBehavior')

In [21]:
def carefulBehavior():
    Policy={}
    for s in actions.keys():
        blueState = s[0]
        redState = s[1]
        if redState + 1 == blueState or redState == blueState + 5 or redState + 6 == blueState or redState + 7 == blueState:
            if blueState < 6:
                Policy[s] = "down"
            else:
                Policy[s] = "stay"
        elif redState + 2 == blueState:
            if blueState < 6:
                Policy[s] = "down"
            else:
                Policy[s] = "up"
        else:
            if blueState < 6:
                Policy[s] = "left"
            else:
                Policy[s] = "up"
    return Policy

careful_policy = carefulBehavior()

In [22]:
careful_intensity = np.array(list(policy_eval(careful_policy).values())).reshape(12,12)
# f, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(careful_intensity, vmin=-100, vmax=10, annot=True, linewidths=0.1, cmap="YlGnBu")
# ax.set_title('Value function for: carefulBehavior')

In [23]:
def semiCarefulBehavior():
    Policy={}
    for s in actions.keys():
        blueState = s[0]
        redState = s[1]
        if redState + 1 == blueState:
            if blueState < 6:
                Policy[s] = "down"
            else:
                Policy[s] = "up"
        elif redState + 2 == blueState:
            if blueState < 6:
                Policy[s] = "stay"
            else:
                Policy[s] = "up"
        elif redState == blueState + 5 or redState + 6 == blueState or redState + 7 == blueState:
            if redState == 11 and blueState == 6:
                Policy[s] = "up"
            else:
                Policy[s] = "stay"
        else:
            if blueState < 6:
                Policy[s] = "left"
            else:
                Policy[s] = "up"
    return Policy

semiCareful_policy = semiCarefulBehavior()

In [24]:
semiCareful_intensity = np.array(list(policy_eval(semiCareful_policy).values())).reshape(12,12)
# f, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(semiCareful_intensity, vmin=-100, vmax=10, annot=True, linewidths=0.1, cmap="YlGnBu")
# ax.set_title('Value function for: carefulBehavior')

In [25]:
def semiAggressiveBehavior():
    Policy={}
    for s in actions.keys():
        blueState = s[0]
        redState = s[1]
        if redState + 1 == blueState:
            if blueState < 6:
                Policy[s] = "stay"
            else:
                Policy[s] = "up"
        elif redState == blueState + 5 or redState + 6 == blueState or redState + 7 == blueState:
            if blueState < 6:
                Policy[s] = "left"
            else:
                Policy[s] = "stay"
        else:
            if blueState < 6:
                Policy[s] = "left"
            else:
                Policy[s] = "up"
    return Policy

semiAggressive_policy = semiAggressiveBehavior()

In [26]:
semiAggressive_intensity = np.array(list(policy_eval(semiAggressive_policy).values())).reshape(12,12)
# f, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(semiAggressive_intensity, vmin=-100, vmax=10, annot=True, linewidths=0.1, cmap="YlGnBu")
# ax.set_title('Value function for: semiAggressiveBehavior')

In [27]:
def aggressiveBehavior():
    Policy={}
    for s in actions.keys():
        blueState = s[0]
        redState = s[1]
        if blueState > 5: #blue is in bX
            Policy[s] = "up"
        else:
            Policy[s] = "left"
    return Policy

aggressive_policy = aggressiveBehavior()

In [28]:
aggressive_intensity = np.array(list(policy_eval(aggressive_policy).values())).reshape(12,12)
# f, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(aggressive_intensity, vmin=-100, vmax=10, annot=True, linewidths=0.1, cmap="YlGnBu")
# ax.set_title('Value function for: aggressiveBehavior')

In [29]:
# diff = pd.DataFrame({'Real': df.groupby(['behavior']).mean()['blueScore'],
#                       'Prediction': [aggressive_intensity[5][0], careful_intensity[5][0], randomBehavior_intensity[5][0] , semiAggressive_intensity[5][0]]},
#                      index= df.groupby(['behavior']).mean()['blueScore'].index)
# diff['different'] = abs(diff['Real']-diff['Prediction'])
# diff

## Value iteration with taking into account the previous state:

In [30]:
class OutOfBoard(Exception):
    pass

In [31]:
def get_pos_after_action(s, a):
    if a == 'U' or a == "up":
        if(s[0] == 'a'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + s[1]
    if a == 'D' or a == "down":
        if(s[0] == 'b'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "b" + s[1]
    if a == 'L' or a == "left":
        if(s[0] == 'b' or s[1] == '1'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + str(int(s[1])-1)
    if a == 'R' or a == "right":
        if(s[0] == 'b' or s[1] == '6'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + str(int(s[1])+1)
    if a == 'S' or a == "stay":
        return s

In [32]:
def get_next_state(c_s):
    n_s = c_s.split()
    blueAction = n_s[3]
    blueState = n_s[2]
    redAction = n_s[1]
    redState = n_s[0]
    
    newPosBlue = get_pos_after_action(blueState, blueAction)
    newPosRed = get_pos_after_action(redState, redAction)
    
    return newPosBlue + ' ' + newPosRed

In [33]:
s_r = df['log'].apply(pd.Series).iloc[:,1:]
s_r.columns = s_r.columns.astype(str)
s_r.set_index( df['index'], inplace=True)
log_red = flat_table.normalize(s_r)
log_red = log_red.dropna(how='all')
log_red.set_index(log_red['index'], inplace=True)
log_red = log_red.drop('index', axis=1)

log_len = int(len(log_red.columns)/2)+1
for i in range(1,log_len):
    log_red[str(i)] = log_red[str(i)+'.red'] + ' ' + log_red[str(i)+'.blue']
    log_red = log_red.drop([str(i)+'.red', str(i)+'.blue'],axis=1)
    for substr in ['from: ', ', action:']:
        log_red[str(i)] = log_red[str(i)].apply(lambda x: x.replace(substr, '', 2) if type(x) != float else x)
    
log_len = len(log_red.columns)
for i in range(1,log_len):
    log_red[str(i) + '&' + str(i+1)] = log_red[str(i)] + '&' + log_red[str(i+1)]
    log_red[str(i) + '&' + str(i+1)] = log_red[str(i) + '&' + str(i+1)].apply(lambda x: x.rsplit(' ', 2)[0] if type(x) != float else x)
log_red = log_red.drop([str(i) for i in range(1,log_len+1)],axis=1)

log_red_copy = log_red.copy(deep=True)

log_red = pd.DataFrame(log_red.to_numpy().flatten(), columns=['state'])
log_red = log_red.dropna()

log_red['action'] = log_red['state'].apply(lambda x: x.rsplit('&', 1)[1].rsplit(' ', 1)[1])
log_red['state'] = log_red['state'].apply(lambda x: x.rsplit('&', 1)[0])

log_red['nxt'] = log_red['state'].apply(get_next_state)
log_red['state'] = log_red['state'].str.split().apply(reversed).apply(' '.join)

log_red['value']=1

for a in ['right', 'stay', 'up', 'down', 'left']:
    log_red['state'] = log_red['state'].apply(lambda x: x.replace(a+' ', '', 2))

log_red['state'] = log_red['state'] + ' nxt: ' + log_red['nxt']
log_red = log_red.drop('nxt',axis=1)

log_red = log_red.groupby(['state', 'action']).count()
log_red.reset_index(inplace=True)
log_red = log_red.set_index('state')

log_red

,action,value
state,,
a1 a2 nxt: a1 a2,right,1
a1 a2 nxt: a1 a3,down,1
a1 a2 nxt: a1 a3,right,24
a1 a2 nxt: a1 a3,stay,1
a1 a2 nxt: a1 b2,stay,1
...,...,...
b6 b5 nxt: b6 a5,down,4
b6 b5 nxt: b6 a5,right,25
b6 b5 nxt: b6 a5,stay,3


In [34]:
result = pd.DataFrame(index=log_red.index.drop_duplicates(), columns=['right', 'stay', 'up', 'down'])
# result.reset_index(level=0, inplace=True)
for action in ['right', 'stay', 'up', 'down']:
    result[action] = log_red[log_red['action'] == action]['value']
result = result.fillna(0)
result.reset_index(inplace=True)
result.sort_index()

,state,right,stay,up,down
0,a1 a2 nxt: a1 a2,1.0,0.0,0.0,0.0
1,a1 a2 nxt: a1 a3,24.0,1.0,0.0,1.0
2,a1 a2 nxt: a1 b2,0.0,1.0,1.0,0.0
3,a1 a3 nxt: a1 a3,1.0,6.0,0.0,3.0
4,a1 a3 nxt: a1 a4,93.0,0.0,0.0,0.0
...,...,...,...,...,...
255,b6 b1 nxt: b6 a1,2.0,0.0,0.0,0.0
256,b6 b1 nxt: b6 b1,0.0,2.0,0.0,0.0
257,b6 b4 nxt: a6 a4,1.0,0.0,0.0,0.0
258,b6 b5 nxt: b6 a5,25.0,3.0,0.0,4.0


In [35]:
# laplace smoving:
for i in range(1,7):
    result.loc[result['state'].str.endswith('a'+str(i)), ['stay', 'right', 'down']] += 1
    
for i in range(1,7):
    result.loc[result['state'].str.endswith('b'+str(i)), ['stay', 'up']] += 1

result = result.set_index('state')
result

,right,stay,up,down
state,,,,
a1 a2 nxt: a1 a2,2.0,1.0,0.0,1.0
a1 a2 nxt: a1 a3,25.0,2.0,0.0,2.0
a1 a2 nxt: a1 b2,0.0,2.0,2.0,0.0
a1 a3 nxt: a1 a3,2.0,7.0,0.0,4.0
a1 a3 nxt: a1 a4,94.0,1.0,0.0,1.0
...,...,...,...,...
b6 b1 nxt: b6 a1,3.0,1.0,0.0,1.0
b6 b1 nxt: b6 b1,0.0,3.0,1.0,0.0
b6 b4 nxt: a6 a4,2.0,1.0,0.0,1.0


#### Convert it to Probability table:

In [36]:
pd.options.display.float_format = "{:,.2f}".format
result_probs = result.div(result.sum(axis=1), axis=0)

#### Save data on computer for the policy iteration:

In [37]:
result_probs.to_json('data/probs2.json' , orient='index', indent=4)

In [64]:
def numberOfStepsRemaining(state, agent):
    ((i,j),(l,k)) = state
    index = i if agent == 0 else j
    moreSteps = index
    if index > 5:
        moreSteps = moreSteps+1

    return moreSteps

In [65]:
numberOfStepsRemaining(((3,4),(9,10)),1)

4

In [40]:
def possibleFutureStates(s):
    n_l = []
    for a in actions[s]:
#         print('s: ', adapt_state((3,2)))
#         print(a)
        n = nextStateDistibution(s, a)
        n_l.extend([elm for elm in n])
#         print('n: ', n)
#         print('n_l: ', n_l)
    return n_l

possibleStatesForState = {}
for s in all_states:
    if s in Policy:
        poss_states = possibleFutureStates(s)
        possibleStatesForState[s] = [s for s in poss_states]

In [41]:
'''==================================================
Initial set up                              version 2
=================================================='''

#Hyperparameters
SMALL_ENOUGH = 0.0001
GAMMA = 0.99

#Define all states
all_states=[]
for prev_state in possibleStatesForState:
    poss_states = possibleStatesForState[prev_state]
    for curr_state in poss_states:
        # current state: (i,j), previous state: (l,k)
        all_states.append((curr_state,prev_state))
all_states.append(((5,0),(-10, -10)))

#Define rewards for all states
rewards = {}
for ((i,j),(l,k)) in all_states:
    if(i == 0): # blue agent in a1
        rewards[((i,j),(l,k))] = richToDestination
    elif(i == j): # crushed
        rewards[((i,j),(l,k))] = crushing
    else:
        rewards[((i,j),(l,k))] = step

#Dictionnary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {}
for ((i,j),(l,k)) in all_states:
    if i != 0 and i != j: #if s=((i, j),(l, k)) is not an "end" state
        actions[((i,j),(l,k))] = ("stay", "left", "down") if (i < 6) else ("up", "stay")

probs = json.load(open('data/probs2.json',))

#Define an initial policy
Policy={}
for s in actions.keys():
    Policy[s] = np.random.choice(actions[s])
    
#Define initial value function 
V={}
for ((i,j),(l,k)) in all_states:
    if ((i,j),(l,k)) in actions.keys():
        V[((i,j),(l,k))] = step
    if(i == 0): # blue agent in a1
        V[((i,j),(l,k))] = richToDestination
    if(i == j): # crushed
        V[((i,j),(l,k))] = crushing

In [42]:
def nextStateDistibution2(s, a):
    # given state s and action a, return distibution of possibles new states by probs
    nxt = {}
#     print('s: ', s)
#     print('a: ', a)
    bluePossition = newPosGivenAction(s[0], a, 0)
#     print('bluePossition: ', bluePossition)
    
    # red possible action given state
    if s[0][1] == 5:
        actions=["stay"]
    elif s[0][1] < 6:
        actions=["right","stay", "down"]
    else:
        actions=["stay", "up"]
#     print('actions: ', actions)
    for r_a in actions:
        redPossition = newPosGivenAction(s[0], r_a, 1)
#         print('r_a: ', r_a)
#         print('redPossition: ', redPossition)
        newState = ((bluePossition, redPossition),s[0])
#         print('newState: ', newState)
        try:
            nxt[newState] = probs[adapt_state(s[1]) + ' nxt: ' + adapt_state(s[0])][r_a]
        except KeyError:
#             print("Warning! State not found: assume random behavior. ", s[0])
            nxt[newState] = randomProbForAction(s[0])
    assert abs(sum(nxt.values()) - 1.0) < 0.0001, "probs not sum to 1.0. \nstate: " + str(s) + " action: " + str(a) + " nxt: " + str(nxt) + "\nsum of probs: " + str(sum(nxt.values()))
    return nxt

In [43]:
pp.pprint(nextStateDistibution2(((0, 4),(1, 4)), "stay"))
pp.pprint(nextStateDistibution2(((4, 1),(5, 0)), "left"))

{((0, 4), (0, 4)): 0.25, ((0, 5), (0, 4)): 0.5, ((0, 10), (0, 4)): 0.25}
{((3, 1), (4, 1)): 0.030651341,
 ((3, 2), (4, 1)): 0.8352490421,
 ((3, 7), (4, 1)): 0.1340996169}


In [44]:
(adapt_state((1, 8)), adapt_state((1, 8)))

('a2 b3', 'a2 b3')

In [45]:
(adapt_state((1, 2)), adapt_state((1, 8)))

('a2 a3', 'a2 b3')

In [46]:
s = ((1, 8), (2, 8))
print((adapt_state((1, 8)), adapt_state((2, 8))))
print(adapt_state(s[1]) + ' nxt: ' + adapt_state(s[0]))
probs[adapt_state(s[1]) + ' nxt: ' + adapt_state(s[0])]

('a2 b3', 'a3 b3')
a3 b3 nxt: a2 b3


{'right': 0.0, 'stay': 0.1212121212, 'up': 0.8787878788, 'down': 0.0}

In [47]:
pp.pprint(nextStateDistibution2(s, "stay"))

{((1, 2), (1, 8)): 0.8787878788, ((1, 8), (1, 8)): 0.1212121212}


In [48]:
'''==================================================
Value Iteration
=================================================='''

iteration = 1

while True:
    biggest_change = 0
    for s in all_states:            
        if s in Policy:
            
            old_v = V[s]
            new_v = np.NINF
            
            for a in actions[s]:
#                 print("actions: ", actions[s])
#                 print("action: ", a)
#                 print("state: ", s)
                nxt = nextStateDistibution2(s, a)
#                 print("nxt:", nxt)
                
                v=0
                #Calculate the value
                for next_state, prob in nxt.items():
#                     print("next_state: ", next_state)
#                     print("prob: ", prob)
#                     print('V[next_state]: ', V[next_state])
                    assert rewards[s] == -1, "rewards is not -1"
                    v += prob * (rewards[s] + GAMMA * V[next_state])
#                 print('v: ', v)
#                 print()
                if v > new_v: #Is this the best action so far? If so, keep it
                    new_v = v
                    Policy[s] = a

           #Save the best of all actions for the state                                
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

            
   #See if the loop should stop now         
    if biggest_change < SMALL_ENOUGH:
        print('converge after: ' + str(iteration) + ' iterations')
        break
    iteration += 1
    
    #Print every 100 iterations
#     if(iteration % 100 == 0):
#         print(iteration, ') ', V[(5, 0)]) # V([a6,a1])

converge after: 917 iterations


In [49]:
def convertStates(Policy):
    P_convert= {}
    V_convert = {}
    for s,v in V.items():
        V_convert[adapt_state(s[0]) + ',' + adapt_state(s[1])] = v
    
#     print()
    for s,p in Policy.items():
        P_convert[adapt_state(s[0]) + ',' + adapt_state(s[1])] = p
    
#     pp.pprint(V_convert)
#     pp.pprint(P_convert)
    return P_convert, V_convert

In [50]:
# print the results:
print("iterations: ", iteration)
optimal_policy_v2, optimal_V_v2 = convertStates(Policy)

iterations:  917


In [51]:
# intensity = np.array(list(V.values())).reshape(1, 693)
# f, ax = plt.subplots(figsize=(2^16, 2^16))
# sns.heatmap(intensity, annot=True, linewidths=0.1, cmap="YlGnBu")

In [52]:
optimal_policy_v2

{'a2 a1,a2 a1': 'down',
 'a2 b1,a2 a1': 'stay',
 'b2 a2,a2 a1': 'up',
 'b2 a1,a2 a1': 'stay',
 'b2 b1,a2 a1': 'up',
 'a2 a4,a2 a3': 'left',
 'a2 a3,a2 a3': 'left',
 'a2 b3,a2 a3': 'left',
 'b2 a4,a2 a3': 'up',
 'b2 a3,a2 a3': 'up',
 'b2 b3,a2 a3': 'up',
 'a2 a5,a2 a4': 'left',
 'a2 a4,a2 a4': 'left',
 'a2 b4,a2 a4': 'left',
 'b2 a5,a2 a4': 'up',
 'b2 a4,a2 a4': 'up',
 'b2 b4,a2 a4': 'up',
 'a2 a6,a2 a5': 'left',
 'a2 a5,a2 a5': 'left',
 'a2 b5,a2 a5': 'left',
 'b2 a6,a2 a5': 'up',
 'b2 a5,a2 a5': 'up',
 'b2 b5,a2 a5': 'up',
 'a2 a6,a2 a6': 'left',
 'b2 a6,a2 a6': 'up',
 'a2 b1,a2 b1': 'stay',
 'a2 a1,a2 b1': 'left',
 'b2 b1,a2 b1': 'up',
 'b2 a1,a2 b1': 'stay',
 'a2 b2,a2 b2': 'left',
 'b2 a2,a2 b2': 'up',
 'a2 b3,a2 b3': 'left',
 'a2 a3,a2 b3': 'left',
 'b2 b3,a2 b3': 'up',
 'b2 a3,a2 b3': 'up',
 'a2 b4,a2 b4': 'left',
 'a2 a4,a2 b4': 'left',
 'b2 b4,a2 b4': 'up',
 'b2 a4,a2 b4': 'up',
 'a2 b5,a2 b5': 'left',
 'a2 a5,a2 b5': 'left',
 'b2 b5,a2 b5': 'up',
 'b2 a5,a2 b5': 'up',
 'a2 b6,

In [53]:
def policy_eval2(policy):
    V={}
    for ((i,j),(l,k)) in all_states:
        if ((i,j),(l,k)) in actions.keys():
            V[((i,j),(l,k))] = step
        if(i == 0): # blue agent in a1
            V[((i,j),(l,k))] = richToDestination
        if(i == j): # crashed
            V[((i,j),(l,k))] = crushing
    while True:
        biggest_change = 0
        for s in all_states:
            if s in Policy:
                old_v = V[s]
                nxt = nextStateDistibution2(s, policy[s])
                v=0
                #Calculate the value
                for next_state, prob in nxt.items():
                    assert rewards[s] == -1, "rewards[s] is not -1"
                    v += prob * (rewards[s] + GAMMA * V[next_state])
        
                #Save the best of all actions for the state                                
                V[s] = v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        #See if the loop should stop now         
        if biggest_change < SMALL_ENOUGH:
            return V

In [54]:
a = policy_eval2(Policy)
a

{((1, 1), (1, 0)): -100,
 ((1, 0), (1, 0)): 4.891475049933639,
 ((1, 6), (1, 0)): 9.547481521549456,
 ((0, 1), (1, 0)): 30,
 ((0, 0), (1, 0)): -100,
 ((0, 6), (1, 0)): 30,
 ((7, 1), (1, 0)): -4.440249999999999,
 ((7, 0), (1, 0)): 6.879724389758994,
 ((7, 6), (1, 0)): 11.69852234150672,
 ((1, 3), (1, 2)): 28.7,
 ((1, 2), (1, 2)): 28.7,
 ((1, 8), (1, 2)): 28.7,
 ((0, 3), (1, 2)): 30,
 ((0, 2), (1, 2)): 30,
 ((0, 8), (1, 2)): 30,
 ((7, 3), (1, 2)): 27.413,
 ((7, 2), (1, 2)): 27.4130000009471,
 ((7, 8), (1, 2)): 27.413,
 ((1, 4), (1, 3)): 28.7,
 ((1, 3), (1, 3)): 28.7,
 ((1, 9), (1, 3)): 28.7,
 ((0, 4), (1, 3)): 30,
 ((0, 3), (1, 3)): 30,
 ((0, 9), (1, 3)): 30,
 ((7, 4), (1, 3)): 27.412999999999997,
 ((7, 3), (1, 3)): 27.412999999999997,
 ((7, 9), (1, 3)): 27.413,
 ((1, 5), (1, 4)): 28.7,
 ((1, 4), (1, 4)): 28.7,
 ((1, 10), (1, 4)): 28.7,
 ((0, 5), (1, 4)): 30,
 ((0, 4), (1, 4)): 30,
 ((0, 10), (1, 4)): 30,
 ((7, 5), (1, 4)): 27.413,
 ((7, 4), (1, 4)): 27.412999999999997,
 ((7, 10), (1, 4)

In [55]:
Policy

{((1, 0), (1, 0)): 'down',
 ((1, 6), (1, 0)): 'stay',
 ((7, 1), (1, 0)): 'up',
 ((7, 0), (1, 0)): 'stay',
 ((7, 6), (1, 0)): 'up',
 ((1, 3), (1, 2)): 'left',
 ((1, 2), (1, 2)): 'left',
 ((1, 8), (1, 2)): 'left',
 ((7, 3), (1, 2)): 'up',
 ((7, 2), (1, 2)): 'up',
 ((7, 8), (1, 2)): 'up',
 ((1, 4), (1, 3)): 'left',
 ((1, 3), (1, 3)): 'left',
 ((1, 9), (1, 3)): 'left',
 ((7, 4), (1, 3)): 'up',
 ((7, 3), (1, 3)): 'up',
 ((7, 9), (1, 3)): 'up',
 ((1, 5), (1, 4)): 'left',
 ((1, 4), (1, 4)): 'left',
 ((1, 10), (1, 4)): 'left',
 ((7, 5), (1, 4)): 'up',
 ((7, 4), (1, 4)): 'up',
 ((7, 10), (1, 4)): 'up',
 ((1, 5), (1, 5)): 'left',
 ((7, 5), (1, 5)): 'up',
 ((1, 6), (1, 6)): 'stay',
 ((1, 0), (1, 6)): 'left',
 ((7, 6), (1, 6)): 'up',
 ((7, 0), (1, 6)): 'stay',
 ((1, 7), (1, 7)): 'left',
 ((7, 1), (1, 7)): 'up',
 ((1, 8), (1, 8)): 'left',
 ((1, 2), (1, 8)): 'left',
 ((7, 8), (1, 8)): 'up',
 ((7, 2), (1, 8)): 'up',
 ((1, 9), (1, 9)): 'left',
 ((1, 3), (1, 9)): 'left',
 ((7, 9), (1, 9)): 'up',
 ((7, 

In [56]:
def extend_policy(policy):
    # take policy without velocity and convert it to policy with velocity
    policy_with_velocity = {}
    for ((i,j),(l,k)) in actions.keys():
        policy_with_velocity[((i,j),(l,k))] = policy[(i,j)]
    return policy_with_velocity

In [57]:
careful_eval_v2 = policy_eval2(extend_policy(careful_policy))
random_eval_v2 = policy_eval2(extend_policy(random_policy))
aggressive_eval_v2 = policy_eval2(extend_policy(aggressive_policy))
semiAggressive_eval_v2 = policy_eval2(extend_policy(semiAggressive_policy))
semiCareful_eval_v2 = policy_eval2(extend_policy(semiCareful_policy))
valueIteration_eval_v2 = policy_eval2(extend_policy(valueIteration_policy))
# [((5,0),(-10,-10))]

NameError: name 'valueIteration_policy' is not defined

In [ ]:
real_col = df.groupby(['behavior']).mean()['blueScore'].drop('carefulBehavior')
diff = pd.DataFrame({'Real': real_col,
                      'Prediction': [aggressive_intensity[5][0], randomBehavior_intensity[5][0], semiAggressive_intensity[5][0], semiCareful_intensity[5][0], np.NaN],
                     'Prediction with velocity': [aggressive_eval_v2[((5,0),(-10,-10))], random_eval_v2[((5,0),(-10,-10))], semiAggressive_eval_v2[((5,0),(-10,-10))], semiCareful_eval_v2[((5,0),(-10,-10))], valueIteration_eval_v2[((5,0),(-10,-10))]]},
                     index= real_col.index)
diff['different'] = abs(diff['Real']-diff['Prediction with velocity'])
diff

In [ ]:
# real_col = df.groupby(['behavior']).mean()['blueScore'].append(pd.Series([np.NaN], index=['valueIterationBehavior']))
# diff = pd.DataFrame({'Real': real_col,
#                       'Prediction': [aggressive_intensity[5][0] , careful_intensity[5][0], randomBehavior_intensity[5][0], semiAggressive_intensity[5][0], semiCareful_intensity[5][0], np.NaN, optimal_V['a6 a1']],
#                      'Prediction with velocity': [aggressive_eval_v2[((5,0),(-10,-10))], careful_eval_v2[((5,0),(-10,-10))], random_eval_v2[((5,0),(-10,-10))], semiAggressive_eval_v2[((5,0),(-10,-10))], semiCareful_eval_v2[((5,0),(-10,-10))],  V[((5,0),(-10,-10))], valueIteration_eval_v2[((5,0),(-10,-10))]]},
#                      index= real_col.index)
# diff['different'] = abs(diff['Real']-diff['Prediction with velocity'])
# diff

In [ ]:
# real_col = df.groupby(['behavior']).mean()['blueScore'].append(pd.Series([np.NaN, np.NaN], index=['valueIterationBehavior', 'valueIterationBehavior_v2'])) #.drop('carefulBehavior')
# diff = pd.DataFrame({'Real': real_col,
#                       'Prediction': [aggressive_intensity[5][0], randomBehavior_intensity[5][0] , semiAggressive_intensity[5][0], semiCareful_intensity[5][0], np.NaN, optimal_V['a6 a1']],
#                      'Prediction with velocity': [aggressive_eval_v2[((5,0),(-10,-10))], random_eval_v2[((5,0),(-10,-10))] , semiAggressive_eval_v2[((5,0),(-10,-10))], semiCareful_eval_v2[((5,0),(-10,-10))],  V[((5,0),(-10,-10))], valueIteration_eval_v2[((5,0),(-10,-10))]]},
#                      index= real_col.index)
# diff['different'] = abs(diff['Real']-diff['Prediction with velocity'])
# diff

In [ ]:
# valueIteration_eval_v2[((5,0),(-10,-10))]

In [ ]:
V[((5,0),(-10,-10))] # this is from the value iteration

In [ ]:
a[((5,0),(-10,-10))] # this is from the eval policy